In [1]:
import os
import gc
import sys
import glob
import numpy as np
import pandas as pd
import netCDF4 as nc
from datetime import datetime, timedelta
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import matplotlib.ticker as mticker

In [2]:
# To use PLUMBER2_GPP_common_utils, change directory to where it exists
os.chdir('/g/data/w97/mm3972/scripts/PLUMBER2/LSM_GPP_PLUMBER2')
from PLUMBER2_GPP_common_utils import *

In [3]:
# Path of PLUMBER 2 dataset
PLUMBER2_path      = "/g/data/w97/mm3972/data/PLUMBER2/"
PLUMBER2_flux_path = "/g/data/w97/mm3972/data/Fluxnet_data/Post-processed_PLUMBER2_outputs/Nc_files/Flux/"
PLUMBER2_met_path  = "/g/data/w97/mm3972/data/Fluxnet_data/Post-processed_PLUMBER2_outputs/Nc_files/Met/"
site_names, IGBP_types, clim_types, model_names = load_default_list()

### Plot diurnal cycle for each site

In [5]:
model_colors = set_model_colors()

#### Get model list

In [6]:
PLUMBER2_path_site = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_GPP_PLUMBER2/nc_files/AR-SLu.nc"
f                  = nc.Dataset(PLUMBER2_path_site, mode='r')
model_list         = f.variables['NEE_models'][:]
model_list         = model_list.tolist()
model_list.append('obs')

In [ ]:
model_list

In [7]:
site_name        = 'AU-Tum'
PLUMBER2_path    = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_GPP_PLUMBER2/txt/process1_NEE_all_sites.csv"
PLUMBER2_qc_path = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_GPP_PLUMBER2/txt/NEE_quality_control_all_sites.csv"

var_output       = pd.read_csv(PLUMBER2_path)
qc_output        = pd.read_csv(PLUMBER2_qc_path)
var_output       = var_output[~np.isnan(qc_output['NEE_qc'])] 

In [15]:
for site_name in site_names:
    var_masked    = var_output[var_output['site_name'][:] == site_name]
    print(np.sum(var_output['model_QUINCY'] < -999))
    
    fig, ax       = plt.subplots(figsize=[10, 7])

    for model_in in model_list:

        var_name          = get_header(model_in)+model_in
        var_diurnal_cycle = var_masked.groupby(['hour']).mean(numeric_only=True)
        if model_in == 'NoahMPv401' or model_in == 'GFDL' or model_in == 'STEMMUS-SCOPE':
            sct               = ax.plot(var_diurnal_cycle[var_name], lw=2.0, color=model_colors[model_in], alpha=0.9, label=model_in)
        else:
            sct               = ax.plot(var_diurnal_cycle[var_name]*(-1), lw=2.0, color=model_colors[model_in], alpha=0.9, label=model_in)

    # ax.set_ylabel("Gross Primary Production (g C m$\mathregular{^{-1}}$ h$\mathregular{^{-1}}$)", fontsize=12)
    ax.set_ylabel("Net ecosystem exchange of CO2 (g C m$\mathregular{^{-1}}$ h$\mathregular{^{-1}}$)", fontsize=12)

    ax.legend(fontsize=8,frameon=False)

    fig.savefig("./plots/diurnal_cycle_NEE_"+site_name,bbox_inches='tight',dpi=300)

0


0